<a href="https://colab.research.google.com/github/mshumer/gpt-investor/blob/main/Claude_Memecoin_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created as a fun experiment by [Matt Shumer](https://twitter.com/mattshumer_).

Claude-Memecoin-Analyst is an educational/informational tool. It should not be considered as financial advice or a substitute for professional investment guidance.

In [ ]:
# Pick your coins here
coins = ["Dogecoin", "Shiba Inu", "Floki"]

# Put your API keys here
ANTHROPIC_API_KEY = "YOUR KEY HERE"  # Replace with your Anthropic API key
SERP_API_KEY = "YOUR KEY HERE"  # Replace with your SERP API key

In [ ]:
!pip install newspaper3k

In [ ]:
import requests
from bs4 import BeautifulSoup
import newspaper
from newspaper import Article
from datetime import datetime
import ast

def get_search_results(search_term):
    url = f"https://serpapi.com/search.json?q={search_term}&api_key={SERP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data['organic_results']

def select_relevant_urls(search_results):
    system_prompt = "You are a crypto analyst assistant. From the given search results, select the URLs that seem most relevant and informative for analyzing the sentiment of the coin."
    search_results_text = "\n".join([f"{i+1}. {result['link']}" for i, result in enumerate(search_results)])
    messages = [
        {"role": "user", "content": f"Search Results:\n{search_results_text}\n\nPlease select the numbers of the URLs that seem most relevant and informative for analyzing the sentiment of the coin. Respond with the numbers in a Python-parseable list, separated by commas."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 200,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']
    numbers = ast.literal_eval(response_text)
    relevant_indices = [int(num) - 1 for num in numbers]
    relevant_urls = [search_results[i]['link'] for i in relevant_indices]
    return relevant_urls

def get_article_text(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

def get_sentiment_analysis(coin, article_texts):
    system_prompt = "You are a crypto sentiment analysis assistant. Analyze the sentiment of the given news articles for the coin and provide a summary of the overall sentiment. Be measured and discerning. You are a skeptical investor."
    combined_text = "\n\n".join(article_texts)
    messages = [
        {"role": "user", "content": f"Coin: {coin}\n\nNews Articles:\n{combined_text}\n\nPlease analyze the sentiment of the news articles for the coin and provide a summary of the overall sentiment. Be measured and discerning. You are a skeptical investor."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 2000,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    sentiment_analysis = response.json()['content'][0]['text']
    return sentiment_analysis

def predict_price_movement(coin, sentiment_analysis):
    system_prompt = "You are a crypto price prediction assistant. Based on the sentiment analysis, predict whether the price of the coin will go up or down. Provide a brief explanation for your prediction."
    messages = [
        {"role": "user", "content": f"Coin: {coin}\n\nSentiment Analysis:\n{sentiment_analysis}\n\nBased on the sentiment analysis, predict whether the price of the coin will go up or down. Provide a brief explanation for your prediction."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 450,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    prediction = response.json()['content'][0]['text']
    return prediction

def rank_coins(coin_data):
    system_prompt = "You are a crypto investment advisor. Based on the sentiment analysis and price predictions for each coin, rank the coins from most attractive to least attractive for investment. Provide a brief rationale for your ranking."
    coin_data_text = "\n\n".join([f"Coin: {coin}\nSentiment Analysis:\n{data['sentiment']}\nPrice Prediction:\n{data['prediction']}" for coin, data in coin_data.items()])
    messages = [
        {"role": "user", "content": f"Coin Data:\n{coin_data_text}\n\nBased on the sentiment analysis and price predictions for each coin, rank the coins from most attractive to least attractive for investment. Provide a brief rationale for your ranking."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 1000,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    ranking = response.json()['content'][0]['text']
    return ranking

def determine_investment(coin_data, ranking):
    system_prompt = "You are a crypto investment advisor. Based on the sentiment analysis, price predictions, and ranking of the coins, provide an overall determination of which coin(s) to invest in. Explain your reasoning."
    coin_data_text = "\n\n".join([f"Coin: {coin}\nSentiment Analysis:\n{data['sentiment']}\nPrice Prediction:\n{data['prediction']}" for coin, data in coin_data.items()])
    messages = [
        {"role": "user", "content": f"Coin Data:\n{coin_data_text}\n\nRanking:\n{ranking}\n\nBased on the sentiment analysis, price predictions, and ranking of the coins, provide an overall determination of which coin(s) to invest in. Explain your reasoning."},
    ]
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 1000,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    investment_decision = response.json()['content'][0]['text']
    return investment_decision

# Get today's date
today = datetime.now().strftime("%Y-%m-%d")
print('Analyzing the following coins:', coins)

coin_data = {}

# Perform analysis for each coin
for coin in coins:
    print(f"\nAnalyzing {coin} for {today}...")

    # Generate search query
    search_query = f"{coin} news {today}"

    # Get search results
    search_results = get_search_results(search_query)

    # Select relevant URLs
    relevant_urls = select_relevant_urls(search_results)

    # Get article text from relevant URLs
    article_texts = []
    for url in relevant_urls:
        try:
            text = get_article_text(url)
            article_texts.append(text)
        except:
            pass

    # Get sentiment analysis
    sentiment_analysis = get_sentiment_analysis(coin, article_texts)
    print(f"\nSentiment Analysis for {coin}:")
    print(sentiment_analysis)

    # Predict price movement
    prediction = predict_price_movement(coin, sentiment_analysis)
    print(f"\nPrice Prediction for {coin}:")
    print(prediction)

    coin_data[coin] = {
        'sentiment': sentiment_analysis,
        'prediction': prediction
    }

    print('\n\n')

# Rank the coins
ranking = rank_coins(coin_data)
print("Ranking of Coins:")
print(ranking)

# Determine which coin(s) to invest in
investment_decision = determine_investment(coin_data, ranking)
print("\nInvestment Decision:")
print(investment_decision)